# Export ONNX

In [ ]:
from glob import glob
from pathlib import Path
from face_alignment import align
from inference import load_pretrained_model, to_input

architecture = 'ir_50'
pretrained_path = '/home/s3-subtitle-download/pretrained_models/AdaFace/adaface_ir50_ms1mv2.ckpt'
model = load_pretrained_model(architecture, pretrained_path)

In [ ]:
path = glob('_test/*.jpg')[0]
aligned_rgb_img = align.get_aligned_face(path)
bgr_input = to_input(aligned_rgb_img)
bgr_input.shape

In [ ]:
import torch
ONNX_PATH = 'adaface.onnx'

dummy_input = torch.randn(bgr_input.shape)

# エクスポート時に動的軸を設定 (バッチサイズの次元を動的に)
torch.onnx.export(model, dummy_input, ONNX_PATH, 
                  input_names=['input'], 
                  output_names=['output'], 
                  dynamic_axes={'input' : {0 : 'batch_size'},  # バッチサイズの次元を動的に
                                'output' : {0 : 'batch_size'}})

In [ ]:
import onnxruntime as ort

# ONNXランタイムセッションを作成
ort_session = ort.InferenceSession(ONNX_PATH)
input_name = ort_session.get_inputs()[0].name

input_data = bgr_input.numpy()
output = ort_session.run(None, {input_name: input_data})
